<h1 align="center">Lab 2:  Sexism Identification in Twitter</h1>
<h2 align="center">Session 1. Machine Learning and Feature Engineering</h2>

<h3 style="display:block; margin-top:5px;" align="center">Natural Language and Information Retrieval</h3>
<h3 style="display:block; margin-top:5px;" align="center">Degree in Data Science</h3>
<h3 style="display:block; margin-top:5px;" align="center">2024-2025</h3>    
<h3 style="display:block; margin-top:5px;" align="center">ETSInf. Universitat Politècnica de València</h3>
<br>

### Put your names here

- Mateusz Król
- (replace with another name)

In [ ]:
# Reading the entire dataset for both languages and considering only the hard labels. In this lab we do not address the sexism identification task from a Learning with Disagreement (LwD) perspective.

from readerEXIST2025 import EXISTReader

reader_train = EXISTReader("EXIST2025_training.json")
reader_dev = EXISTReader("EXIST2025_dev.json")

EnTrainTask1, EnDevTask1 = reader_train.get(lang="EN", subtask="1"), reader_dev.get(lang="EN", subtask="1")
EnTrainTask2, EnDevTask2 = reader_train.get(lang="EN", subtask="2"), reader_dev.get(lang="EN", subtask="2")

SpTrainTask1, SpDevTask1 = reader_train.get(lang="ES", subtask="1"), reader_dev.get(lang="ES", subtask="1")
SpTrainTask2, SpDevTask2 = reader_train.get(lang="ES", subtask="2"), reader_dev.get(lang="ES", subtask="2")


In [ ]:
print(type(reader_train))

<class 'readerEXIST2025.EXISTReader'>


In [ ]:
reader_train.get(lang="EN", subtask="2")[1]

,text
1,Writing a uni essay in my local pub with a cof...
2,@UniversalORL it is 2021 not 1921. I dont appr...
5,According to a customer I have plenty of time ...
6,"So only 'blokes' drink beer? Sorry, but if you..."
10,#EverydaySexism means women usually end up in ...
...,...
3253,@ShefVaidya Ma'am if I say that you look like ...
3255,idk why y’all bitches think having half your a...
3256,This has been a part of an experiment with @Wo...
3257,"""Take me already"" ""Not yet. You gotta be ready..."


# ENGLISH

## Preprocessing

In [ ]:
import pandas as pd
import re

In [ ]:
# COMPLETE IF YOU WANT TO DO TEXT PREPROCESSING

web_re = re.compile(r"https?:\/\/[^\s]+", re.U)
user_re = re.compile(r"(@\w+\-?(?:\w+)?)", re.U)
hashtag_re = re.compile(r"(#\w+\-?(?:\w+)?)", re.U)

def preprocess(text):
    # COMPLETE
    text = pd.Series(text)
    text = text.str.replace(user_re, "", regex=True)
    text = text.str.replace(web_re, "", regex=True)
    text = text.str.replace(hashtag_re, "", regex=True).str.lower()
    return text

In [ ]:
print(EnTrainTask1)

(1       200002
2       200003
5       200006
6       200007
7       200008
         ...  
3255    203256
3256    203257
3257    203258
3258    203259
3259    203260
Name: id, Length: 2870, dtype: object, 1       Writing a uni essay in my local pub with a cof...
2       @UniversalORL it is 2021 not 1921. I dont appr...
5       According to a customer I have plenty of time ...
6       So only 'blokes' drink beer? Sorry, but if you...
7       New to the shelves this week - looking forward...
                              ...                        
3255    idk why y’all bitches think having half your a...
3256    This has been a part of an experiment with @Wo...
3257    "Take me already" "Not yet. You gotta be ready...
3258    @clintneedcoffee why do you look like a whore?...
3259    ik when mandy says “you look like a whore” i l...
Name: text, Length: 2870, dtype: object, 1       YES
2       YES
5       YES
6       YES
7        NO
       ... 
3255    YES
3256    YES
3257    YES
3258    

In [ ]:
EnTrainTask1_new = preprocess(EnTrainTask1[1])
EnDevTask1_new = preprocess(EnDevTask1[1])
EnTrainTask2_new = preprocess(EnTrainTask2[1])
EnDevTask2_new = preprocess(EnDevTask2[1])
SpTrainTask1_new = preprocess(SpTrainTask1[1])
SpDevTask1_new = preprocess(SpDevTask1[1])
SpTrainTask2_new = preprocess(SpTrainTask2[1])
SpDevTask2_new = preprocess(SpDevTask2[1])

print(SpTrainTask1_new)

3260     ignora al otro, es un capullo.el problema con...
3261     si comicsgate se parece en algo a gamergate p...
3262     lee sobre gamergate, y como eso ha cambiado l...
3264       entonces como así es el mercado lo mejor no...
3265     aaah sí. andrew dobson. el que se dedicó a ec...
                              ...                        
6914    que envidia me dan las dos zorras estas, yo de...
6915    los hombres sieeeempre dicen que si fueran muj...
6916     estas zorras qls siempre preocupadas de hasta...
6917    a pesar de que convenga que las zorras jueguen...
6919     aaaaaaa ¿quién te manda socializar con nosotr...
Name: text, Length: 3194, dtype: object


In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
# # Preprocessing
# lemmatizer = WordNetLemmatizer()
# stop_words = set(stopwords.words('english'))

# def clean_text(text):
#     text = text.lower()  # Lowercasing
#     text = re.sub(r'\W', ' ', text)  # Remove punctuation
#     text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
#     words = word_tokenize(text)  # Tokenization
#     words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]  # Lemmatization & stopword removal
#     return ' '.join(words)

# # Apply to dataset
# EnTrainTask1_cleaned = [clean_text(text) for text in EnTrainTask1[1]]
# EnDevTask1_cleaned = [clean_text(text) for text in EnDevTask1[1]]

# print(EnTrainTask1_cleaned[:5])  # See cleaned texts


In [ ]:
!pip install emoji

In [ ]:
import re
import pandas as pd
import emoji

# Regular expressions for cleaning
web_re = re.compile(r"https?:\/\/[^\s]+", re.U)  # Remove URLs
user_re = re.compile(r"(@\w+\-?(?:\w+)?)", re.U)  # Remove mentions
hashtag_re = re.compile(r"(#\w+\-?(?:\w+)?)", re.U)  # Remove hashtags
special_chars_re = re.compile(r"[^a-zA-Z0-9\s]", re.U)  # Remove special characters

def preprocess(text):
    text = pd.Series(text)  # Convert input to Pandas Series if it's not already

    text = text.str.replace(user_re, "", regex=True)  # Remove @mentions
    text = text.str.replace(web_re, "", regex=True)  # Remove URLs
    text = text.str.replace(hashtag_re, "", regex=True)  # Remove hashtags
    text = text.apply(lambda x: emoji.replace_emoji(x, replace=""))  # Remove emojis
    text = text.str.replace(special_chars_re, "", regex=True)  # Remove special characters
    text = text.str.lower().str.strip()  # Convert to lowercase and remove extra spaces

    return text.tolist()  # Convert back to a list if needed

# Example usage
sample_tweets = [
    "@user1 OMG! This is crazy 😡 #shockingnews https://t.co/xyz123",
    "Women deserve equal pay! #EqualPayNow 💰",
    "Visit our website: www.example.com #ad"
]

cleaned_tweets = preprocess(sample_tweets)
print(cleaned_tweets)

['omg this is crazy', 'women deserve equal pay', 'visit our website wwwexamplecom']


In [ ]:
EnTrainTask1_cleaned = [preprocess(text) for text in EnTrainTask1[1]]
EnDevTask1_cleaned = [preprocess(text) for text in EnDevTask1[1]]
EnTrainTask2_cleaned = [preprocess(text) for text in EnTrainTask2[1]]
EnDevTask2_cleaned = [preprocess(text) for text in EnDevTask2[1]]
SpTrainTask1_cleaned = [preprocess(text) for text in SpTrainTask1[1]]
SpDevTask1_cleaned = [preprocess(text) for text in SpDevTask1[1]]
SpTrainTask2_cleaned = [preprocess(text) for text in SpTrainTask2[1]]
SpDevTask2_cleaned = [preprocess(text) for text in SpDevTask2[1]]

### Text representation

## Tweet representations (Feature extraction)

### TF-IDF

In [ ]:
# Convert everything to string and remove NaN values
EnTrainTask1_cleaned = [str(text) for text in EnTrainTask1_cleaned if text is not None]
EnDevTask1_cleaned = [str(text) for text in EnDevTask1_cleaned if text is not None]
EnTrainTask2_cleaned = [str(text) for text in EnTrainTask2_cleaned if text is not None]
EnDevTask2_cleaned = [str(text) for text in EnDevTask2_cleaned if text is not None]

SpTrainTask1_cleaned = [str(text) for text in SpTrainTask1_cleaned if text is not None]
SpDevTask1_cleaned = [str(text) for text in SpDevTask1_cleaned if text is not None]
SpTrainTask2_cleaned = [str(text) for text in SpTrainTask2_cleaned if text is not None]
SpDevTask2_cleaned = [str(text) for text in SpDevTask2_cleaned if text is not None]

# Ensure no empty strings (optional)
EnTrainTask1_cleaned = [text for text in EnTrainTask1_cleaned if text.strip() != ""]
EnDevTask1_cleaned = [text for text in EnDevTask1_cleaned if text.strip() != ""]
EnTrainTask1_cleaned = [text for text in EnTrainTask1_cleaned if text.strip() != ""]
EnDevTask1_cleaned = [text for text in EnDevTask1_cleaned if text.strip() != ""]

SpTrainTask1_cleaned = [text for text in SpTrainTask1_cleaned if text.strip() != ""]
SpDevTask1_cleaned = [text for text in SpDevTask1_cleaned if text.strip() != ""]
SpTrainTask2_cleaned = [text for text in SpTrainTask2_cleaned if text.strip() != ""]
SpDevTask2_cleaned = [text for text in SpDevTask2_cleaned if text.strip() != ""]

# Check again
print(type(EnTrainTask1_cleaned[0]))

<class 'str'>


In [ ]:
print(EnTrainTask1_cleaned[:5])

["['writing a uni essay in my local pub with a coffee random old man keeps asking me drunk questions when im trying to concentrate amp ends with good luck but youll just end up getting married and not use it anyway  is alive and well']", "['it is 2021 not 1921 i dont appreciate that on two rides by myself your team member looked behind me and asked the man behind how many in my party not impressed']", "['according to a customer i have plenty of time to go spent the stirling coins he wants to pay me with in derry just like any other woman im sure of it  in retail']", "['so only blokes drink beer sorry but if you arent a bloke you drink wine apparently  alive and well in']", "['new to the shelves this week  looking forward to reading these books']"]


In [ ]:
print(SpTrainTask1_cleaned[:5])  # Sample Spanish tweets after preprocessing

["['ignora al otro es un capulloel problema con este youtuber denuncia el acoso cuando no afecta a la gente de izquierdas por ejemplo en su video sobre el gamergate presenta como normal el acoso que reciben fisher anita o zey cuando hubo hasta amenazas de bomba']", "['si comicsgate se parece en algo a gamergate pues muy bien por el acoso y si se est haciendo un sabotaje porque hay personajes que no os gustan entonces gracias por darme la razn sois unos lloricas ofendidos']", "['lee sobre gamergate y como eso ha cambiado la manera en la cual nos comunicamos en el internet los fanboys de halo estn txicos pero los fanboys de otras comunidadesjuegos tambin han querido coger pauta con eso']", "['entonces como as es el mercado lo mejor no es hacer algo para cambiarlo y seguir alimentando el machismo en los consumidores en lugar apoyar a gente como las vctimas del gamergateacerca de lo otro el tenan implica un imperativo entonces no entiendo lo del buscaban']", "['aaah s andrew dobson el que 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder

# Convert labels to numerical format
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(EnTrainTask1[2])  # Binary labels

# TF-IDF Vectorization
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X_train_tfidf = vectorizer.fit_transform(EnTrainTask1_cleaned)  # Training text data

In [ ]:
# print(len(EnTrainTask1_cleaned))  # Should be 2870
# print(len(y_train))  # Should also be 2870

In [ ]:
# Convert labels to numerical format
label_encoder2 = LabelEncoder()
y_train2 = label_encoder2.fit_transform(EnTrainTask2[2])  # Binary labels

# TF-IDF Vectorization
vectorizer2 = TfidfVectorizer(stop_words='english', max_features=5000)
X_train_tfidf2 = vectorizer2.fit_transform(EnTrainTask2_cleaned)  # Training text data

X_dev_tfidf2 = vectorizer2.transform(EnDevTask2_cleaned)
y_dev2 = label_encoder2.transform(EnDevTask2[2])

### Word2Vec

In [ ]:
# !pip install gensim

In [ ]:
# import spacy
# nlp = spacy.load("es_core_news_md")  # Load Spanish model

# def get_embedding(text):
#     return nlp(text).vector  # Extract dense vector representation

# X_train_emb = [get_embedding(text) for text in SpTrainTask1_cleaned]
# X_dev_emb = [get_embedding(text) for text in SpDevTask1_cleaned]


In [ ]:
# # Obtaining a representation for the train and dev subsets in both tasks

# # COMPLETE
# # import gensim
# from gensim.models import Word2Vec
# from nltk.tokenize import word_tokenize

# # Tokenize sentences
# tokenized_texts = [word_tokenize(text.lower()) for text in SpTrainTask1_cleaned]

# # Train Word2Vec model
# w2v_model = Word2Vec(sentences=tokenized_texts, vector_size=100, window=5, min_count=2, workers=4)

# # Convert text data to vectors
# def get_avg_word_vector(text, model, vector_size):
#     words = word_tokenize(text.lower())
#     vectors = [model.wv[word] for word in words if word in model.wv]
#     return sum(vectors) / len(vectors) if vectors else [0] * vector_size

# X_train_w2v = [get_avg_word_vector(text, w2v_model, 100) for text in SpTrainTask1_cleaned]
# X_test_w2v = [get_avg_word_vector(text, w2v_model, 100) for text in SpDevTask1[1]]

# print(len(X_train_w2v), len(X_train_w2v[0]))  # Output: (num_samples, 100)


## Learning Models

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import f1_score, classification_report

# Train Logistic Regression
lr = LogisticRegression()
lr.fit(X_train_tfidf, y_train)

# Predict on development set
X_dev_tfidf = vectorizer.transform(EnDevTask1_cleaned)
y_dev = label_encoder.transform(EnDevTask1[2])

y_pred = lr.predict(X_dev_tfidf)

f1_positive	= f1_score(y_dev, y_pred, pos_label=1)
print("Subtask 1")
print(f"F1-score (Positive	Class):	{f1_positive}")
print(classification_report(y_dev, y_pred))

Subtask 1
F1-score (Positive	Class):	0.6827794561933535
              precision    recall  f1-score   support

           0       0.74      0.90      0.81       250
           1       0.82      0.58      0.68       194

    accuracy                           0.76       444
   macro avg       0.78      0.74      0.75       444
weighted avg       0.77      0.76      0.76       444



In [ ]:
# Train Logistic Regression
lr2 = LogisticRegression()
lr2.fit(X_train_tfidf2, y_train2)

# Predict on development set
X_dev_tfidf2 = vectorizer2.transform(EnDevTask2_cleaned)
y_dev2 = label_encoder2.transform(EnDevTask2[2])

y_pred2 = lr2.predict(X_dev_tfidf2)

f1_positive	= f1_score(y_dev2, y_pred2, average = 'macro')
print("Subtask 2")
print(f"F1-score (Positive	Class):	{f1_positive}")
print(classification_report(y_dev2, y_pred2))

Subtask 2
F1-score (Positive	Class):	0.32091290172333486
              precision    recall  f1-score   support

           0       0.60      1.00      0.75        85
           1       0.00      0.00      0.00        28
           2       0.80      0.12      0.21        33

    accuracy                           0.61       146
   macro avg       0.47      0.37      0.32       146
weighted avg       0.53      0.61      0.49       146



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
from sklearn import svm
from sklearn.metrics import f1_score, classification_report
###########################SUBTASK	1##################################
#Using	the	SVC	with	default	settings
clf1_en	= svm.SVC()
clf1_en.fit(X_train_tfidf, y_train)
predicted1 = clf1_en.predict(X_dev_tfidf)
# print("Predicted task1\t-->\t",	predicted1)
# print("GroundTruth task1\t-->\t", y_dev)
#Computing	the	F1	Score	for	the	positive	class	(sexism),	subtask	1
print("Subtask 1")
f1_positive	= f1_score(y_dev,	predicted1,	pos_label=1)
print(f"F1-score (Positive	Class):	{f1_positive}")
#Obtainig a detailed classification	report
report	= classification_report(y_dev, predicted1, digits=4)
print(report)

Subtask 1
F1-score (Positive	Class):	0.6966966966966966
              precision    recall  f1-score   support

           0     0.7443    0.9080    0.8180       250
           1     0.8345    0.5979    0.6967       194

    accuracy                         0.7725       444
   macro avg     0.7894    0.7530    0.7574       444
weighted avg     0.7837    0.7725    0.7650       444



In [ ]:
###########################SUBTASK	2##################################
clf2_en	= svm.SVC()
clf2_en.fit(X_train_tfidf2, y_train2)
predicted2 = clf2_en.predict(X_dev_tfidf2)
# print("Predicted task2\t-->\t",	predicted2)
# print("GroundTruth task2\t-->\t", EnDevTask2[2])
#Computing	the	macro-average F1 for the second	subtask
f1_macro = f1_score(y_dev2, predicted2,	average='macro')
print("Subtask 2")
print(f"F1-score (Macro-Averaged): {f1_macro}")
#Obtainig a detailed classification	report
report = classification_report(y_dev2, predicted2, digits=4)
print(report)

Subtask 2
F1-score (Macro-Averaged): 0.24531024531024528
              precision    recall  f1-score   support

           0     0.5822    1.0000    0.7359        85
           1     0.0000    0.0000    0.0000        28
           2     0.0000    0.0000    0.0000        33

    accuracy                         0.5822       146
   macro avg     0.1941    0.3333    0.2453       146
weighted avg     0.3389    0.5822    0.4285       146



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# COMPLETE

In [ ]:
from sklearn.neural_network	import MLPClassifier
from sklearn.metrics	import f1_score,	classification_report
###########################SUBTASK	1##################################
#Using	the	MLP	with	default	settings
clf1_en	=	MLPClassifier(random_state=1,	max_iter=300)
clf1_en.fit(X_train_tfidf,	y_train)
predicted1	=	clf1_en.predict(X_dev_tfidf)
# print("Predicted	task1\t-->\t",	predicted1)
# print("GroundTruth	task1\t-->\t",	y1_dev)
#Computing	the	F1	Score	for	the	positive	class	(sexism),	subtask	1
f1_positive	=	f1_score(y_dev,	predicted1,	pos_label=1)
print("Subtask 1")
print(f"F1-score	(Positive	Class):	{f1_positive}")
#Obtainig	a	detailed	classification	report
report	=	classification_report(y_dev, predicted1,	digits=4)
print(report)
###########################SUBTASK	2##################################
clf2_en	=	MLPClassifier(random_state=1,	max_iter=300)
clf2_en.fit(X_train_tfidf2,	y_train2)
predicted2	=	clf2_en.predict(X_dev_tfidf2)
# print("Predicted	task2\t-->\t",	predicted2)
# print("GroundTruth	task2\t-->\t",	y2_dev)
#Computing	the	macro-average	F1	for	the	second	subtask
f1_macro	=	f1_score(y_dev2,	predicted2,	average='macro')
print("Subtask 2")
print(f"F1-score	(Macro-Averaged):	{f1_macro}")
#Obtainig	a	detailed	classification	report
report	=	classification_report(y_dev2, predicted2,	digits=4)
print(report)

Subtask 1
F1-score	(Positive	Class):	0.6049046321525886
              precision    recall  f1-score   support

           0     0.6937    0.7520    0.7217       250
           1     0.6416    0.5722    0.6049       194

    accuracy                         0.6734       444
   macro avg     0.6677    0.6621    0.6633       444
weighted avg     0.6710    0.6734    0.6707       444

Subtask 2
F1-score	(Macro-Averaged):	0.4642297695075139
              precision    recall  f1-score   support

           0     0.6759    0.8588    0.7565        85
           1     0.4000    0.2143    0.2791        28
           2     0.4348    0.3030    0.3571        33

    accuracy                         0.6096       146
   macro avg     0.5036    0.4587    0.4642       146
weighted avg     0.5685    0.6096    0.5747       146



## Show Results

In [ ]:
#ref - y_true
#pred - y_pred

In [ ]:
#EXAMPLE 1 TASK1 and TASK2 FOR ENGLISH
from sklearn import svm
from sklearn.metrics import f1_score, classification_report

def show_subtask1(ref, pred):
    # Computing the F1 Score for the positive class (sexism), subtask 1
    f1_positive = f1_score(ref, pred, pos_label=1, zero_division=0)
    # Obtainig a detailed classification report
    report = classification_report(ref, pred, digits=4, zero_division=0)
    print("\nSUBTASK 1")
    print(f"F1-score (Positive Class): {f1_positive}")
    print(report)

def show_subtask2(ref, pred):
    # Computing the macro-average F1 for the second subtask
    f1_macro = f1_score(ref, pred, average='macro', zero_division=0)
    # Obtainig a detailed classification report
    report = classification_report(y2_dev,predicted2, digits=4, zero_division=0)
    print("\nSUBTASK 2")
    print(f"F1-score (Macro-Averaged): {f1_macro}")
    print(report)


###########################SUBTASK 1##################################

# COMPLETE

###########################SUBTASK 2##################################

# COMPLETE



SUBTASK 1
F1-score (Positive Class): 0.7192982456140351
              precision    recall  f1-score   support

           0     0.7601    0.9000    0.8242       250
           1     0.8311    0.6340    0.7193       194

    accuracy                         0.7838       444
   macro avg     0.7956    0.7670    0.7717       444
weighted avg     0.7911    0.7838    0.7784       444


SUBTASK 1
F1-score (Positive Class): 0.7333333333333333
              precision    recall  f1-score   support

           0     0.7944    0.7880    0.7912       250
           1     0.7296    0.7371    0.7333       194

    accuracy                         0.7658       444
   macro avg     0.7620    0.7626    0.7622       444
weighted avg     0.7661    0.7658    0.7659       444


SUBTASK 2
F1-score (Macro-Averaged): 0.285668928840735
              precision    recall  f1-score   support

           0     0.5986    1.0000    0.7489        85
           1     0.0000    0.0000    0.0000        28
           2 

# SPANISH

## Preprocessing

In [ ]:
# COMPLETE IF YOU WANT TO DO TEXT PREPROCESSING
# Already made above

## Tweet representations (Feature extraction)

In [ ]:
!pip install stopwordsiso

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 4.3 MB/s eta 0:00:00


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from stopwordsiso import stopwords  # Install via: pip install stopwordsiso
spanish_stopwords = stopwords("es")  # Get Spanish stop words

In [ ]:
# Obtaining a representation for the train and dev subsets in both tasks

# COMPLETE
# Convert labels to numerical format
label_encoder3 = LabelEncoder()
y_train3 = label_encoder3.fit_transform(SpTrainTask1[2])  # Binary labels

# TF-IDF Vectorization
vectorizer3 = TfidfVectorizer(stop_words=list(spanish_stopwords),max_features=5000)
X_train_tfidf3 = vectorizer3.fit_transform(SpTrainTask1_cleaned)  # Training text data

X_dev_tfidf3 = vectorizer3.transform(SpDevTask1_cleaned)
y_dev3 = label_encoder3.transform(SpDevTask1[2])

# Convert labels to numerical format
label_encoder4 = LabelEncoder()
y_train4 = label_encoder4.fit_transform(SpTrainTask2[2])  # Binary labels

# TF-IDF Vectorization
vectorizer4 = TfidfVectorizer(stop_words=list(spanish_stopwords), max_features=5000)
X_train_tfidf4 = vectorizer4.fit_transform(SpTrainTask2_cleaned)  # Training text data

X_dev_tfidf4 = vectorizer4.transform(SpDevTask2_cleaned)
y_dev4 = label_encoder4.transform(SpDevTask2[2])

## Learning Models

In [ ]:
# COMPLETE

lr3 = LogisticRegression()
lr3.fit(X_train_tfidf3, y_train3)

y_pred3 = lr3.predict(X_dev_tfidf3)

f1_positive	= f1_score(y_dev3, y_pred3, pos_label=1)
print("Subtask 1")
print(f"F1-score (Positive	Class):	{f1_positive}")
print(classification_report(y_dev3, y_pred3))

Subtask 1
F1-score (Positive	Class):	0.7551867219917012
              precision    recall  f1-score   support

           0       0.71      0.83      0.76       229
           1       0.82      0.70      0.76       261

    accuracy                           0.76       490
   macro avg       0.76      0.76      0.76       490
weighted avg       0.77      0.76      0.76       490



In [ ]:
lr4 = LogisticRegression()
lr4.fit(X_train_tfidf4, y_train4)

y_pred4 = lr4.predict(X_dev_tfidf4)

f1_positive	= f1_score(y_dev4, y_pred4, average='macro')
print("Subtask 2")
print(f"F1-score (Positive	Class):	{f1_positive}")
print(classification_report(y_dev4, y_pred4))

Subtask 2
F1-score (Positive	Class):	0.34456992283079235
              precision    recall  f1-score   support

           0       0.60      0.99      0.75       116
           1       0.67      0.12      0.20        51
           2       0.33      0.05      0.09        40

    accuracy                           0.59       207
   macro avg       0.53      0.39      0.34       207
weighted avg       0.56      0.59      0.48       207



In [ ]:
###########################SUBTASK	1##################################
#Using	the	SVC	with	default	settings
clf1_en	= svm.SVC()
clf1_en.fit(X_train_tfidf3, y_train3)
predicted1 = clf1_en.predict(X_dev_tfidf3)
# print("Predicted task1\t-->\t",	predicted1)
# print("GroundTruth task1\t-->\t", y_dev)
#Computing	the	F1	Score	for	the	positive	class	(sexism),	subtask	1
print("Subtask 1")
f1_positive	= f1_score(y_dev3,	predicted1,	pos_label=1)
print(f"F1-score (Positive	Class):	{f1_positive}")
#Obtainig a detailed classification	report
report	= classification_report(y_dev3, predicted1, digits=4)
print(report)

Subtask 1
F1-score (Positive	Class):	0.7436974789915967
              precision    recall  f1-score   support

           0     0.6945    0.8341    0.7579       229
           1     0.8233    0.6782    0.7437       261

    accuracy                         0.7510       490
   macro avg     0.7589    0.7561    0.7508       490
weighted avg     0.7631    0.7510    0.7504       490



In [ ]:
clf2_en	= svm.SVC()
clf2_en.fit(X_train_tfidf4, y_train4)
predicted2 = clf2_en.predict(X_dev_tfidf4)
# print("Predicted task2\t-->\t",	predicted2)
# print("GroundTruth task2\t-->\t", EnDevTask2[2])
#Computing	the	macro-average F1 for the second	subtask
f1_macro = f1_score(y_dev4, predicted2,	average='macro')
print("Subtask 2")
print(f"F1-score (Macro-Averaged): {f1_macro}")
#Obtainig a detailed classification	report
report = classification_report(y_dev4, predicted2, digits=4)
print(report)

Subtask 2
F1-score (Macro-Averaged): 0.2960966357192772
              precision    recall  f1-score   support

           0     0.5743    1.0000    0.7296       116
           1     1.0000    0.0588    0.1111        51
           2     0.5000    0.0250    0.0476        40

    accuracy                         0.5797       207
   macro avg     0.6914    0.3613    0.2961       207
weighted avg     0.6648    0.5797    0.4454       207



In [ ]:
from sklearn.neural_network	import MLPClassifier
from sklearn.metrics	import f1_score,	classification_report
###########################SUBTASK	1##################################
#Using	the	MLP	with	default	settings
clf1_en	=	MLPClassifier(random_state=1,	max_iter=300)
clf1_en.fit(X_train_tfidf3,	y_train3)
predicted1	=	clf1_en.predict(X_dev_tfidf3)
# print("Predicted	task1\t-->\t",	predicted1)
# print("GroundTruth	task1\t-->\t",	y1_dev)
#Computing	the	F1	Score	for	the	positive	class	(sexism),	subtask	1
f1_positive	=	f1_score(y_dev3,	predicted1,	pos_label=1)
print("Subtask 1")
print(f"F1-score	(Positive	Class):	{f1_positive}")
#Obtainig	a	detailed	classification	report
report	=	classification_report(y_dev3, predicted1,	digits=4)
print(report)
###########################SUBTASK	2##################################
clf2_en	=	MLPClassifier(random_state=1,	max_iter=300)
clf2_en.fit(X_train_tfidf4,	y_train4)
predicted2	=	clf2_en.predict(X_dev_tfidf4)
# print("Predicted	task2\t-->\t",	predicted2)
# print("GroundTruth	task2\t-->\t",	y2_dev)
#Computing	the	macro-average	F1	for	the	second	subtask
f1_macro	=	f1_score(y_dev4,	predicted2,	average='macro')
print("Subtask 2")
print(f"F1-score	(Macro-Averaged):	{f1_macro}")
#Obtainig	a	detailed	classification	report
report	=	classification_report(y_dev4, predicted2,	digits=4)
print(report)

Subtask 1
F1-score	(Positive	Class):	0.6965376782077393
              precision    recall  f1-score   support

           0     0.6538    0.7424    0.6953       229
           1     0.7435    0.6552    0.6965       261

    accuracy                         0.6959       490
   macro avg     0.6987    0.6988    0.6959       490
weighted avg     0.7016    0.6959    0.6960       490

Subtask 2
F1-score	(Macro-Averaged):	0.46408250355618774
              precision    recall  f1-score   support

           0     0.6689    0.8534    0.7500       116
           1     0.3600    0.1765    0.2368        51
           2     0.4412    0.3750    0.4054        40

    accuracy                         0.5942       207
   macro avg     0.4900    0.4683    0.4641       207
weighted avg     0.5488    0.5942    0.5570       207



## Report

## 1. Preprocessing Steps
To ensure high-quality input for the classification models, several preprocessing techniques were applied to the tweets:
- **Removal of Unnecessary Elements:** Hashtags, mentions, URLs, special characters, and emojis were removed.
- **Text Normalization:** Extra spaces were eliminated, and all text was converted to lowercase to maintain uniformity.
- **Stopwords Removal:** English and Spanish stopwords were removed to reduce noise and improve model focus on meaningful words.

## 2. Text Representations
To convert text data into numerical format, **TF-IDF (Term Frequency-Inverse Document Frequency)** was used:
- **Feature Extraction:** The TF-IDF vectorizer was applied with `max_features=5000` to limit vocabulary size.
- **Stopwords Handling:** Both English and Spanish stopwords were excluded.
- **Normalization:** Text was lowercased to ensure case-insensitive tokenization.

## 3. Classification Models and Hyperparameters
Three machine learning models were used for training and evaluation:

### Binary Classification (Sexist vs. Non-Sexist Tweets)
#### Support Vector Machine (SVM)
- **F1-score:** 0.69 (English), 0.73 (Spanish)

#### Logistic Regression
- **F1-score:** 0.75 (Spanish), 0.68 (English)

#### Multi-layer Perceptron (MLP) Neural Network
- **F1-score:** 0.60 (English), 0.69 (Spanish)

### Multiclass Classification (Sexist Tweet Categorization: DIRECT, REPORTED, JUDGEMENTAL)
#### Multi-layer Perceptron (MLP) Neural Network
- **F1-score:** 0.46 (English & Spanish)

#### Support Vector Machine (SVM)
- **F1-score:** Below 0.40

## 4. Observations and Conclusions
- **Binary classification achieved better performance** than multiclass classification, especially for Spanish tweets.
- **SVM performed best for English binary classification**, while **logistic regression was best for Spanish binary classification**.
- **MLP outperformed other models in multiclass classification**, but scores were generally low (~0.46 F1-score), indicating the difficulty in distinguishing between sexist categories.
- **Future Improvements:** Using deep learning models like **BERT embeddings** instead of TF-IDF could enhance contextual understanding and improve classification performance.

### Final Conclusion
Logistic Regression and SVM were the most effective for binary classification, while MLP worked best for multiclass classification, although overall multiclass performance remained low.



## Show Results

In [ ]:
###########################SUBTASK 1##################################

# COMPLETE

###########################SUBTASK 2##################################

# COMPLETE



SUBTASK 1
F1-score (Positive Class): 0.7370600414078675
              precision    recall  f1-score   support

           0     0.6903    0.8079    0.7445       229
           1     0.8018    0.6820    0.7371       261

    accuracy                         0.7408       490
   macro avg     0.7461    0.7449    0.7408       490
weighted avg     0.7497    0.7408    0.7405       490


SUBTASK 1
F1-score (Positive Class): 0.7613412228796844
              precision    recall  f1-score   support

           0     0.7213    0.7686    0.7442       229
           1     0.7846    0.7395    0.7613       261

    accuracy                         0.7531       490
   macro avg     0.7529    0.7540    0.7528       490
weighted avg     0.7550    0.7531    0.7533       490


SUBTASK 2
F1-score (Macro-Averaged): 0.27065897065897065
              precision    recall  f1-score   support

           0     0.5779    0.9914    0.7302       116
           1     0.2500    0.0196    0.0364        51
           